In [1]:
import pandas as pd
import numpy as np
import re
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

## DON'T TOUCH DUMMY

In [18]:
#Soup for listings pages
headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:95.0) Gecko/20100101 Firefox/95.0",
            "X-Amzn-Trace-Id": "Root=1-61c3ff21-6d7180d97e6822561d378dd9"}

No_pages = 30
book_soups = []

for pageNo in range(1 , No_pages+1):
    r = requests.get('https://www.amazon.ca/s?i=digital-text&rh=n%3A2980423011&fs=true&page='+str(pageNo)+'&qid=1640233972&ref=sr_pg_'+str(pageNo), headers=headers)
    content = r.content
    soup = BeautifulSoup(content)
    for d in soup.findAll('div', class_ = "s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16"):
        links = d.find('a', class_ = "a-link-normal s-no-outline")
        link = links['href']
        book_r = requests.get('https://www.amazon.ca'+link, headers=headers)
        book_content = book_r.content
        book_soup = BeautifulSoup(book_content)
        book_soups.append(book_soup)

## GOOD TO TOUCH

In [29]:
def get_data(s):
    data_list = []
    
    #book title
    title = s.find('span', id = 'productTitle')
    data_list.append(title.text if title is not None else np.nan)
    
    #author (allows for more than one)
    author_list = s.findAll('span', class_ = 'author notFaded')
    authors = []
    for a in author_list:
        author = a.find('a', class_ = 'a-link-normal contributorNameID')
        if author is not None:
            authors.append(author.text)
    data_list.append(authors if authors is not [] else np.nan)
    
    #price
    price = s.find('span', id = 'kindle-price')
    data_list.append(price.text if price is not None else np.nan)
    
    #kindle unlimited 
    ku_icon = s.find('i', class_ = 'a-icon a-icon-kindle-unlimited a-icon-medium')
    data_list.append(1 if ku_icon is not None else 0)
    
    #ranking
    product_details = s.find('div', id="detailBullets_feature_div")
    if product_details is not None:
        detail_groups = product_details.findAll('ul', class_ = "a-unordered-list a-nostyle a-vertical a-spacing-none detail-bullet-list")
        ranks = detail_groups[1].find('span', class_ = "a-list-item")
        data_list.append(ranks.text if ranks is not None else np.nan)
    
     #overall rating
        rating = detail_groups[2].find('span', class_ = 'a-icon-alt')
        data_list.append(rating.text if rating is not None else np.nan)
    
    #number of ratings
    rating_count = s.find('span', attrs={'id':'acrCustomerReviewText', 'class':'a-size-base'})
    data_list.append(rating_count.text if rating_count is not None else np.nan)
    
    #blurb
    blurb = s.find('div', id = 'bookDescription_feature_div')
    data_list.append(blurb.text if blurb is not None else np.nan)
    
    #reviews
    reviews = s.find('div', class_="a-section a-spacing-large reviews-content filterable-reviews-content celwidget")
    
    reviews_ratings = []
    reviews_titles = []
    reviews_texts = []
    
    if reviews is not None:
        reviews_list = reviews.findAll('div', class_="a-section review aok-relative")
    
        for r in reviews_list:
            #get rating from all top reviews
            review_rating = r.find('a', class_ = "a-link-normal")
            reviews_ratings.append(review_rating['title'] if review_rating is not None else np.nan)
        
            #get titles from all top reviews
            review_title = r.find('a', class_ = "a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold")
            reviews_titles.append(review_title.text if review_title is not None else np.nan)
        
            #get text from all top reviews
            review_text = r.find('div', class_ = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content')
            reviews_texts.append(review_text.text if blurb is not None else np.nan)
    
    data_list.append(reviews_ratings if reviews_ratings is not [] else np.nan)
    data_list.append(reviews_titles if reviews_titles is not [] else np.nan)
    data_list.append(reviews_texts if reviews_texts is not [] else np.nan)
    
    return data_list


In [30]:
#loop over all listings
all_data = []
for b in book_soups:
    b_data = get_data(b)
    all_data.append(b_data)

Next I convert the scraped data into a data frame and save a copy as a csv into my repository for safekeeping :)

In [32]:
#Export to csv
book_data = pd.DataFrame(all_data,columns=['Book Name','Author', 'Price', 'KU', 'Rankings','Rating','Customers_Rated','Blurb', 'Reviews_Ratings', 'Reviews_Titles', 'Reviews_Texts' ])
book_data.to_csv('amazon_books.csv', index=False, encoding='utf-8')